In [1]:
import os
import ast
import redis
import matplotlib
import pandas as pd
import seaborn as sns
matplotlib.use('Agg')  # Use the 'Agg' backend for Matplotlib
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from flask import Flask, render_template
from flask_basicauth import BasicAuth

from income import Income
from visualization import vizualize_income_bar, prepare_to_vizualize_secotrs_bar, vizualize_sectors_bar

In [2]:

# data_loader = DataLoader()
# transaction_data, sectors_data, daily_prices, dividends_data = data_loader.run()

In [3]:
load_dotenv()

# Initialize Redis connection
redis_host = os.getenv('REDIS_HOST')
redis_port = int(os.getenv('REDIS_PORT'))
redis_password = os.getenv('REDIS_PASSWORD')

r = redis.Redis(host=redis_host, port=redis_port, password=redis_password)

# Fetch data from Redis
transaction_data_json = r.get('transaction_data')
portfolio_to_plot_json = r.get('portfolio_to_plot')
portfolio_table_json = r.get('portfolio_table')
dividends_data_json = r.get('dividends_data')
growth_data_json = r.get('growth_table')

transaction_data = pd.DataFrame(ast.literal_eval(transaction_data_json.decode('utf-8')))
transaction_data['date'] = pd.to_datetime(transaction_data['date'], unit='ms')
transaction_data['start_payment_date'] = pd.to_datetime(transaction_data['start_payment_date'], unit='ms')
portfolio_table = pd.DataFrame(ast.literal_eval(portfolio_table_json.decode('utf-8')))
portfolio_to_plot = pd.DataFrame(ast.literal_eval(portfolio_to_plot_json.decode('utf-8')))
portfolio_to_plot['Date'] = pd.to_datetime(portfolio_to_plot['Date'], unit='ms')
dividends_data = pd.DataFrame(ast.literal_eval(dividends_data_json.decode('utf-8')))
income = Income(transaction_data, dividends_data)
monthly_income, yearly_income = income.run()
growth_data_str = growth_data_json.decode('utf-8')
growth_data_str = growth_data_str.replace('null', "-1")
growth_data = pd.DataFrame(ast.literal_eval(growth_data_str))
growth_data['start_payment_date'] = pd.to_datetime(growth_data['start_payment_date'], unit='ms')
growth_data.columns = [" ".join([part.capitalize() for part in col.split('_')]) for col in growth_data.columns]

In [4]:
from executive_summery import ExecutiveSummery

In [5]:
#Display
## Excetive Summary
summery = ExecutiveSummery(portfolio_table)
total_return, amount_invested, return_yield = summery.get_total_return()
dividend_yield, yield_on_cost = summery.get_dividend_yield(total_return, amount_invested, yearly_income)
average_dividend_growth = summery.get_average_dividend_growth(growth_data)